# Task 5: Temporal Planning and Reasoning 

### Teammates:

* Kayla Holman
* Lisa R Peng
* Will Parker
* Yue Meng

In [ ]:
# Assignment

#DONE 
# 0. data structure (Yue)
# 1. Build_constraint_graphs (Yue)

#TODO (describe for)
# 2. is_consistency (Kayla)
# 3. build_apsp_graph (Will)
# 4. schedule_under_apsp_graph (Lisa)


## Timeline

### ~~Week 9 - Project Skeleton (Due: 4pm, Oct. 30, Friday)~~
1. ~~Completed the required reading~~
2. ~~Provide a skeleton Jupyter notebook~~
3. ~~Outlining the main ideas of each topic in markup cells~~
4. ~~Commented code cells which include function stubs for the capabilities to be demonstrated~~



### Week 12 - First Draft (Due: 4pm, Nov. 20, Friday)
1. Including detailed walkthroughs of the ideas involved in each topic
2. Working code complementing each section.
    - Implementation of main functions
    - Explanations about the functions


### Week 15 - Final Submission (4pm, Dec. 9, Wednesday)
A final draft incorporating feedback from the TAs

## Problem Background
YOU are an engineer hired by the Generic Space Empire for a project. You have been assigned to a moon
space station, the center of controversy throughout the galaxy.
Maintenance personnel have noticed that several workstations have been acting up, but have assured the
engineering team that “uh, everything’s perfectly all right now”. You, however, suspect something
sinister afoot. Could it be sabotage from the Underdog Rebels?
You know that a document for the space station was recently stolen (many Non-Copyrighted Alien spies
died), in which several attacks taking advantage of design flaws have been documented. Each attack
requires compromising a combination of components, distributed across the workstations.
While you can not dismantle the workstations to directly inspect the components, you can send a team of
diagnosis droids to make observations on the workstations to exactly determine the faulty components. As
you perform diagnosis on each workstation, you can update the odds on which type of attack will occur.
You must do this in a timely manner, so that you can escape before an attack if necessary, and warn the
Big Bad Guy Wearing Black if time permits.


### Temporal Planning and Reasoning
In this task, you will be asked to check whether you have time to warn the Big Bad Guy Wearing Black,
or whether you should just try to escape. You will be given duration information for traversals, diagnosis,
and possible additional actions you might take, as well as timing constraints you must meet in order to
escape on a shuttle or provide warnings. You will be asked to check for the feasibility of alternative plans,
providing a schedule for a feasible plan, and provide an explanation for each infeasible plan.

Inputs:

1. Set of possible actions
2. Set of events
3. Temporal constraints

Outputs:

1. Feasibility?
2. Conflict if infeasible
3. Schedule if feasible
4. Best temporally feasible plan


### Readings
Representing temporal relations, Allen’s temporal algebra and simple temporal networks; temporal consistency, scheduling and dynamic execution. 

Reading: 

* Dechter, Meiri and Pearl, “Temporal Constraint Networks,” – on
Stellar. 
* Optional Reading: Muscettola, Morris and Tsamardinos, “Reformulation Temporal Plans for Efficient
Execution,” – on Stellar.

# Code Section

1. First translate input to multiple constraint graphs
2. Then for each constraint graph:
    1. Check whether it's feasible (and point out where it causes conflict)
    2. Compute its minimal distance graph
    3. Use minimal distance graph to do scheduling
    4. Output the best feasible plan (I guess saving the Big Guy, and also in the shortest time?)

# Input formulation
Define actions $\mathcal{A}$, events $\mathcal{E}$, and temporal constraints: $\mathcal{C} = \{(e_0,e_1,a,t_{start},t_{end})\in \mathcal{E}\times \mathcal{E}\times \mathcal{A}\times \mathbb{R}_{\geq 0}\times \mathbb{R}_{\geq 0}\}$,
which means "starting from event $e_0$ to event $e_1$ via action $a$ will take $[t_{start}, t_{end}]$ minutes".

For example, John goes to work either by car (30-40 minutes) or by bus (60-100 minutes),
and Fred goes to work either by car (20-30 minutes) or carpool (40-50 minutes). They left home at the same time and John arrived at work 10-20 minutes before Fred arrived. Then we will have the actions, events and constraints as following:

Actions: by car, by bus, by carpool, NULL (when no specific action is required)

Events: John left home, John arrived at work, Fred left home, Fred arrived at work

Constraints: 

1. (John left home, John arrived at work, by car, 30, 40)
2. (John left home, John arrived at work, by bus, 60, 100)
3. (Fred left home, Fred arrived at work, by car, 20, 30)
4. (Fred left home, Fred arrived at work, by carpool, 40, 50)
5. (John left home, Fred left home, NULL, 0, 0)
6. (John arrived at work, Fred arrived at work, NULL, 10, 20)

We would like to generate graphs based on those inputs and to see what's the best scheduling. Notice that there are some pairs of events that can have multiple plausible actions (home->work, by bus/by car). 
For each valid assignment of actions between events, we build a constraint graph (a directional graph), where the edge represents the time limit. For example, here we can choose 
(John left home, John arrived at work, by car, 30, 40), (Fred left home, Fred arrived at work, by car, 20, 30), (John left home, Fred left home, NULL, 0, 0) and 
(John arrived at work, Fred arrived at work, NULL, 10, 20) to build a graph.


For each graph we check the consistency, solve the D-graph and find the scheduling.
Finally we compare all the possible schedulings and provide the best one.




## Storyline (v1.0)

When we enter the information room, a message shows on the screen, **"Explosion countdown, 1:00:00"**. We must leave the station!(\[0, 60\])

We need to go together with the big bad guy. But he will not wake up until 44 minutes later, and will only stay awake for 11 minutes (\[44, 55\]).

The path is InfoRoom->Guard->MiddlePoint->BigBadGuy->BusStation (this is a one-direction-path). 
Traversing between adjacent points using drones will take at least 10 minutes, and the battery can only keep for 30 minutes on each path (\[10, 30\]).

The guard controls the gates for BigBadGuy and BusStation. Our hacker code will work on those gates simultaneously.
It will open the gate at BigBadGuy after exact 24 minutes, but can only keep the gate open for 12 minutes. (\[24, 36\])
And it will open the gate at BusStation after exact 34 minutes, but can only keep the gate open for 17 minutes.  (\[34, 51\])

Can we make it? What shall we do?



## Figure

Something like this
![constraints-graph](constraints-graph.png)

In [ ]:
# necessary data structures for APSP representation
class Node:
    def __init__(self, name="default_node"):
        self.name=name
        self.ins=[]
        self.outs=[]
    
    def add_in_edge(self, edge_name):
        self.ins.append(edge_name)
    
    def add_out_edge(self, edge_name):
        self.outs.append(edge_name)


class Edge:
    def __init__(self, name="default_edge", fr="default_node_0", to="default_node_1", cost=-1):
        self.name=name
        self.fr=fr
        self.to=to
        self.cost=cost
    def __str__(self):
        return "name: " + self.name + \
            "\n from: " + self.fr + \
            "\n to: " + self.to + \
            "\n cost: " + str(self.cost)


class Graph:
    def __init__(self):
        self.nodes=dict() # name->"Node object"
        self.edges=dict() # (name x name)->"Edge object"
        self.edge_list=[]
    
    def add_node(self, name):
        node = Node(name)
        self.nodes[name] = node

    def add_edge(self, name=None, fr=None, to=None, cost=None):
        if name is None:
            name = fr+"->"+to
        edge = Edge(name, fr, to, cost)
        if fr not in self.edges:
            self.edges[fr] = dict()
        self.edges[fr][to] = edge
        self.edge_list.append(edge)
    
    def get_node(self, name):
        if name not in self.nodes:
            return None
        return self.nodes[name]

    def has_edge(self, fr, to):
        return (fr in self.edges and to in self.edges[fr])

    def get_edge(self, fr, to):
        if self.has_edge(fr, to):
            return self.edges[fr][to]
        return None
    
    def get_edge_cost(self, fr, to):
        if self.has_edge(fr, to):
            return self.edges[fr][to].cost
        else:
            return None
    
    def get_node_names(self):
        return list(self.nodes.keys())
    

# TODO(yue) design some tests

# Construct the Distance Graph
As a first step towards solving this problem, we will want to translate the provided constraint graph in our problem statement to a weighted (distance) graph. In the distance graph, we create directed arcs into and out of connected nodes. The upper bound from a constraint graph edge pair corresponds to the weight on the outgoing arc from a node in the edge pair, while the lower bound from the constraint graph corresponds to the incoming arc for a node. 

Given a list of actions, events, and constraints for a given constraint problem, the following builds the distance graph using the graph implementation defined above. For the constraints given, the upper bounds of the intervals connect nodes in the forward direction, and lower bounds connect nodes in the backwards direction and are negated. 

In [ ]:
# TODO fill in the data from storyline
graph=Graph()

# add nodes
graph.add_node("A")
graph.add_node("B")
graph.add_node("C")
graph.add_node("D")
graph.add_node("E")

# add edges
graph.add_edge(fr="A", to="B", cost=30)
graph.add_edge(fr="B", to="A", cost=-10)
graph.add_edge(fr="A", to="D", cost=55)
graph.add_edge(fr="D", to="A", cost=-44)
graph.add_edge(fr="A", to="E", cost=60)
graph.add_edge(fr="E", to="A", cost=-0)

graph.add_edge(fr="B", to="C", cost=30)
graph.add_edge(fr="C", to="B", cost=-10)
graph.add_edge(fr="B", to="D", cost=36)
graph.add_edge(fr="D", to="B", cost=-24)
graph.add_edge(fr="B", to="E", cost=51)
graph.add_edge(fr="E", to="B", cost=-34)

graph.add_edge(fr="C", to="D", cost=30)
graph.add_edge(fr="D", to="C", cost=-10)

graph.add_edge(fr="D", to="E", cost=30)
graph.add_edge(fr="E", to="D", cost=-10)

# TODO(yue)
# This is the case that trivial greedy will fail, because we cannot go as fast as possible, instead we need to wait for some opportunities.
# I know one feasible solution is (A=0, A->B=20, B=20, B->C=15, C=35, C->D=10, D=45, D->E=15, E=60)

#TODO(yue) we can first omit this
# def build_distance_graphs(actions, events, constraints):
#     '''
#     Params:
#       actions:
#       events:
#       constraints:
#     Returns:
#       An instance of the Graph class representing the distance graph for the given inputs
#     '''
#     raise NotImplementedError

# Simple baseline

To solve the problem, we need to have a scheduling that can satisfy all the constraints. One simple method for scheduling is to randomly assign values by the order A->B->C->D->E and then make sure they satisify the path-wise constraints.
For example, we assign t=0 to A, and then from A to B has the constraint \[10,30\]

# Checking Consistency of Distance Graph
Given the distance graph, we need to check if there are any negative cycles that will render the problem unsolvable. This can either be done after making the distance graph or after creating the d-graph, but to avoid longer runtimes when there is an inconsistency, we'll do the check before making the d-graph. This will be done using the Bellman-Ford algorithm. The Bellman-Ford algorithm finds single-source shortest paths. We are using it to detect negative cycles in our constraint graph, because a negative cycle would mean our defined constraints are inconsistent with eachother. 

The key idea of Bellman-Ford is that we keep track of the shortest known distance between the start node and every node so far, and iterate through every edge to update those shortest distances every round. We do |V|-1 rounds total to guarantee that we have found the shortest path from the start node to every node because there are |V| number of nodes total. That is a fact of the Bellman-Ford algorithm. By running another round of the whole Bellman-Ford algorithm, if the distance to a node in the graph can be further reduced, we know we have a negative cycle because the first round of Bellman-Ford should've found all shortest paths from the single source already. Therefore the consistency check for the constraint graph is done by running Bellman-Ford twice for a total of O(|V||E|) time complexity.

In [ ]:
import string
def is_consistent(graph):
  '''
  Params:
    graph: An instance of the Graph class representing the distance graph
  Returns:
    False if the graph has negative cycles, True otherwise
  '''
  #check outputted dp of APSP for negative values along the diagonal entries d_ii
  #TODO(yue): another way to check consistency is to use Bellman-Ford algorithm
  #           which gives O(VE) time complexity, as suggested from 
  #           https://stackoverflow.com/questions/16898416/fastest-algorithm-to-detect-if-there-is-negative-cycle-in-a-graph
  # Approach Bellman Ford
  num_nodes = len(graph.nodes)
  edges_list = graph.edge_list
  # D stores shortest distance from start node to every node in the graph
  D = [float('inf')]*num_nodes
  D[0] = 0 #initialize distance to start node
  #relax each edge num_nodes-1 times because we're guaranteed to find shortest path to all nodes after this many iterations
  for i in range(num_nodes-1):
    for edge in edges_list:
      cost = edge.cost
      fr_idx = string.ascii_uppercase.index(edge.fr)
      to_idx = string.ascii_uppercase.index(edge.to)
      if D[fr_idx] + cost < D[to_idx]:
        # update D with shorter path if found
        D[to_idx] = D[fr_idx] + cost
  #at this point we've found shortest paths from start node to every other node in the graph
  #UNLESS there's a negative cycle
  #run another loop to detect if any edges can be relaxed. If so, that node is part of a negative cycle.
  for i in range(num_nodes-1):
    for edge in edges_list:
      cost = edge.cost
      fr_idx = string.ascii_uppercase.index(edge.fr)
      to_idx = string.ascii_uppercase.index(edge.to)
      if D[fr_idx] + cost < D[to_idx]:
        # mark node as part of negative cycle
        D[to_idx] = float('-inf')
  #at this point we can find nodes connected to a negative cycle, or just return if a negative cycle exists
  return float('-inf') not in D
print(is_consistent(graph))


True



# Construct the APSP Complete Directed Graph 

Now that we know that there are no negative cycles, we can begin our search for the temporally feasible solutions. To do this, we'll want to identify the shortest possible path between every pair of nodes (i.e. solve the "all pairs shortest path" (APSP) problem). 


The APSP problem can be solved using the Floyd-Warshall or Johnson algorithm. Floyd-Warshall requires first editing the distance-graph with weights of 0 for edges connecting nodes to themselves, and with weights of +inf (or some very large number) for edges that do not exist (i.e. for nodes that are not connected). Once this edited distance graph is created, F-W works by checking all possible paths from one node to another, and recording the path that has the lowest cost. Since F-W requires checking all V^2 posible combinations of nodes, and requires checking all V possible paths connecting the two nodes, F-W has complexity that is O(V^3). This makes computational cost increase substantially with an increase in the number of nodes (vertices). In contrast, Johnson's algorithm is O(V^2\log V + VE), and is better suited for sparse graphs. 

In [ ]:
def floyd_warshall_shortest_path(graph): 
    #this is Lisa: what's the diff between this fxn and build_apsp_graph?
    #TODO(Yue): there are two ways to setup APSP graph:
    #           1. Floyd-Warshall algorithm (O(V^3)) - fits dense graphs
    #           2. Johnson’s algorithm (O(V^2\log V + VE)) - fits sparse graphs
    #           it would be cool if we can adaptively route the build_apsp_graph
    #           function towards different handlers, based on graph sparsity

    #create a deepcopy of the distance graph to use for APSP d-graph. 
    import copy
    d_graph = copy.deepcopy(graph)
    #start by converting all empty edges in graph to 0 (if to/from nodes are the same) or +inf (if nodes not connected at all)
    for fr in d_graph.nodes.keys():
        for to in d_graph.nodes.keys():
            if d_graph.get_edge_cost(fr,to):
                #if there's already an edge cost, this node is already well defined. 
                continue
            elif fr == to:
                #if both nodes are the same, then the assigned edge cost should be zero. 
                d_graph.add_edge(fr=fr,to=to,cost=0)
                # graph.add_edge(fr = 'A',to = 'A',  0)
            else:
                #if nodes not connected, assign a very large cost to that edge (i.e. 10**8)
                d_graph.add_edge(fr=fr,to=to,cost=10**8)

    # Perform F-W algorithm by looping through each possible pair of nodes, and comparing the cost of the current shortest path to the path that runs through an intermediate (mid) node.
    for mid in d_graph.nodes.keys():
        for fr in d_graph.nodes.keys():
            for to in d_graph.nodes.keys():
                # graph for iteration k is found by comparing current shortest paths to the path that runs through node 'mid'
                newCost = min(d_graph.get_edge_cost(fr,to), d_graph.get_edge_cost(fr, mid) + d_graph.get_edge_cost(mid,to))
                d_graph.add_edge(fr=fr, to=to, cost=newCost)
    #now, the distances on the graph represent the APSP solution. 
    return(d_graph)

In [ ]:
def johnson_shortest_path(graph):
    #O(V^2LogV + VE)
    #uses Dijkstra's and Bellman-Ford as subroutines
    #Dijkstra's finds shortest paths, however cannot deal with negative weight edges
    #Johnson's converts all negative weights to positive, then runs Dijkstra's for every node
    raise NotImplementedError


In [ ]:
def build_apsp_d_graph(graph):
    #May want to implement checker to switch between F-W and Johnson depending on graph sparsity. For now, just routing directly back to F-W. 
    d_graph = floyd_warshall_shortest_path(graph)
    return(d_graph)


In [ ]:
def find_conflicts(graph):
    raise NotImplementedError


# Use Solution by Decomposition to extract a path.

input: APSP d-graph

output: assignment as a list representing sequence of actions with time interval pairs

Now that we've constructed the APSP d-graph, the only thing that is left to do is to extract feasible schedules/paths and find the most temporally feasible plan. 

In [ ]:


# assignment-related functions
def evaluate_assignment(assignment):
    raise NotImplementedError

# assignment-related functions
def assignment_to_string(assignment):
    raise NotImplementedError


In [ ]:
def main(actions, events, constraints):
    graphs = build_constraint_graphs(actions, events, constraints)
    assigment_list = []
    for i,graph in enumerate(graphs):
        if is_consistent(graph):
            apsp_graph = build_apsp_graph(graph)
            assignment = schedule_under_apsp_graph(apsp_graph)
            print("graph-%d is feasbile, and one greedy plan is %s"%(i, assignment))
            assigment_list.append(assignment)
        else:
            conflicts = find_conflicts(graph)
            print("graph-%d is infeasible, and the conflicts are found in %s"%(i, conflicts))

    if len(assignment_list)==0:
        print("No feasible plan can be found")
    else:
        best_assignment = sorted(assignment_list, key=lambda x:evaluate_assignment(x), reversed=True)[0]
        print("The best plan is %s"%(best_assignment))


#TODO do some tests
main([],[],[])


NameError: name 'build_constraint_graphs' is not defined

In [ ]:
graph

In [ ]:
graph